# Performance comparison of option calculations

## European vanilla options

In [1]:
import os
from random import random, seed
import sys
from timeit import timeit

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.offline import iplot

sys.path.append(os.path.normpath(os.path.join(os.getcwd(), os.pardir)))

In [ ]:
from methods.binom_trees import call_eur_binom_tree
from methods.black_scholes import call_eur_bs

### Convergence to the BS call value

In [ ]:
seed(42)
n_steps = [2, 5, 10, 20, 30, 50, 75, 100, 200]
len_steps = len(n_steps)
addons = []
for _ in range(1000):
    s = random() * 50 + 10  # U(10, 60)
    k = s + (random() - 0.5) * 9 
    r = 0.001 + random() / 20  # U(0.001, 0.201)
    sig = 0.01 + random() / 1.5  # U(0.01, 0.67666)
    t = random() * (np.sqrt(sig / r) - 0.01) + 0.01  # U(0.1, sqrt(sig/r)). Above, pu > 1 in the formulas in 13.8
    bs = call_eur_bs(s=s, k=k, r=r, sig=sig, t=t)
    tree = [call_eur_binom_tree(s=s, k=k, r=r, sig=sig, t=t, n=n_i) for n_i in n_steps]
    tree_abs = [i - bs for i in tree]
    tree_rel = [i / max(0.001, bs) for i in tree]
    addons.append(pd.DataFrame({
        "s": [s]*len_steps,
        "k": [k]*len_steps,
        "r": [r]*len_steps,
        "sig": [sig]*len_steps,
        "t": [t]*len_steps,
        "n": n_steps,
        "bs": [bs]*len_steps,
        "tree": tree,
        "ratio": tree_rel,
        "diff": tree_abs,
        
    }))
comparisons = pd.concat(addons, axis=0, ignore_index=True)
comparisons["n"] = comparisons["n"].astype("category")

In [ ]:
px.scatter(comparisons, x="bs", y="tree", color="n", hover_data=["s", "k", "r", "sig", "t"],
           title="Tree vs. BS-Values of European call options for different tree sizes n")

As expected, there seem to be systematic errors for very low n (interesting questions: Why? Is the apparent oscillation obvious from theory?), but after about 20 or so, there are no mor gross outliers.

In [ ]:
fig = px.box(comparisons, x="n", y="ratio",
            title="Ratios between Tree and BS values of European calls")
fig.update_yaxes(range=[0.99, 1.01])
fig.show()

The ratios between the two methods show the expected narrowing of ranges with increasing n. It's notable that with n=30 used in the book, the difference in option prices is still about half a percent, which is not small. Also there are some larger mistakes that occur if the BS value is small.

In [ ]:
fig = px.box(comparisons, x="n", y="diff",
             title="Differences between tree and BS values of European calls")
fig.update_yaxes(range=[-1, 1])
plotly.offline.iplot(fig)

Differences yield a more conciliatory picture, but of course that especially for low values, even small differences can make a significant differences.

### Performance difference

In [ ]:
s = 50
k = 48
r = 0.01
sig = 0.2
t = 5
n_trials = 1000
t_bs = timeit(lambda: call_eur_bs(s=s, k=k, r=r, sig=sig, t=t), number=n_trials)
t_tree = [timeit(lambda: call_eur_binom_tree(s=s, k=k, r=r, sig=sig, t=t, n=n), number=n_trials) for n in n_steps]


In [ ]:
coefs = np.polyfit(n_steps, t_tree, 2)
n_x = 200
x = np.linspace(0, max(n_steps), n_x)
y = coefs[2] + coefs[1] * x + coefs[0] * x ** 2

In [ ]:
bs = go.Scatter(x=x, y=[t_bs / 1000]*n_x, mode="lines", name="BS", marker={"color": "hsl(210, 50%, 50%)"})
quadratic_fit = go.Scatter(x=x, y=y/1000, mode="lines", name="Quadratic fit", marker={"color": "hsl(120, 70%, 50%)"})
actual = go.Scatter(x=n_steps, y=[i / 1000 for i in t_tree], mode="markers", name="Actual runtimes", marker={"size": 10, "color": "hsl(120, 90%, 30%)"})
layout = {
    'showlegend': True,
    "title": "Runtimes of Binomial trees vs Black Scholes"
}

fig = {
    'layout': layout,
    'data': [bs, quadratic_fit, actual],
}

iplot(fig)

At `n=30` trees take about twice as long as BS. While 1000 BS take about a third of a second, the same amount of `n=100`trees take more than 6 seconds. For individual calculations both certainly seem acceptable. As long as the number of executions stays low, this is not considered a problem. In Cases of significant load, a change in programming language may be in order anyway.